# SUBMARINE CABLES PROJECT

## Plotting fault locations given in the Telegeography data

----------------------------------------------------------------------------------------------

### Overview :
### This notebook contains the code for finding out the tentative (most probable) coordinates for given fault descriptions in the provided dataset.

-------

### Overall working involves :

 1.  Using externally sources cable and landing points data
 2.  Each cable is then associated with its landing points
 3.  Parsing of fault descriptions to extract candidate locations
 4.  Matching candidate locations to landing points to get coordinates; or else geocoding is used
 5. Parsing of fault descriptions to extract patterns
 6. These patterns are then used to have a structured understanding of the given descriptiond
 7. Traversing on different segments of a particular cable can be important when metrics such as distance needs to be calculate. So, segment connectivty is found.
 8. Segment connectivity graphs and the maps for each cable and its' segments are plotted
 9. The final inferred location is used to get the tentative coordinates for each of the description


In [ ]:
TELEGEOGRAPHY_FAULT_DATA_FILE_PATH = "telegeography_data_updated.csv"

# here, kindly give the path of the location where the concerned telegeography fault data file is present
# NOTE :::: the given csv file here should contain the fault category too

In [ ]:
import math
import os
import re
import time
import json
import pandas as pd
import geopandas as gpd
import networkx as nx
import matplotlib.pyplot as plt
import folium

from shapely.geometry import LineString, Point
from geopy.distance import geodesic, distance
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
from folium import GeoJson, GeoJsonTooltip, Map, Marker
from difflib import get_close_matches
from collections import deque
import spacy


In [ ]:
fault_data = pd.read_csv(TELEGEOGRAPHY_FAULT_DATA_FILE_PATH)   
fault_data = fault_data[['Cable', 'Fault Location']]
fault_data = fault_data.dropna(subset=['Fault Location'])
fault_data = fault_data[fault_data['Fault Location'].str.strip() != ''] 
fault_data.head(2)


with open("cable-geo.json", "r") as cable_file:
    cable_data = json.load(cable_file)["features"]
    
with open("landing-point-geo.json", "r") as landing_points_file:
    landing_points_data = json.load(landing_points_file)["features"]
    
    
####################
# here, these 2 datasets were obtained from another github repo
# cable-geo.json contains the information about each cable and it's trajectory in the form of coordinates in multistrings
# landing-point-geo.json contains the information about the landing points and the coordinates


In [ ]:
# to check how the fault data looks like
fault_data.head(10)

,Cable,Fault Location
0,PEACE Cable,"in the Red Sea, 1,450km from Zafarana"
2,C-Lion1,Sweeden Economic Zone
4,Asia Pacific Gateway (APG),S1.9 branch
5,Asia Pacific Gateway (APG),"S9 branch, located 143km from Singapore CLS"
6,Darwin-Jakarta-Singapore Cable (DJSC),off the coast of Port Hedland
8,Sweden-Latvia,between Latvian town of Ventspils to Sweden's ...
9,Taiwan Penghu Kinmen Matsu No.2 (TPKM2),between Taiwan and Matsu
10,Taiwan Penghu Kinmen Matsu No.3 (TPKM3),between Taiwan and Matsu
12,Trans-Pacific Express (TPE) Cable System,close to the port of Keelung on Taiwan’s north...
13,Asia Africa Europe-1 (AAE-1),near Qatar


In [ ]:
cable_data
# here, we try to examine the structure of the cable_data

[{'type': 'Feature',
  'properties': {'id': 'tanjung-pandan-sungai-kakap',
   'name': 'Tanjung Pandan-Sungai Kakap',
   'color': '#a84c9c',
   'feature_id': 'tanjung-pandan-sungai-kakap-0',
   'coordinates': [108.00543396858707, -1.1603546988114444]},
  'geometry': {'type': 'MultiLineString',
   'coordinates': [[[109.18222689022609, -0.061391357195038],
     [108.82736530508038, -0.061391357195038],
     [108.67782927819995, -0.179555522582304],
     [107.95935298091241, -1.227571400214666],
     [107.56616014384186, -2.148726042101377],
     [107.5300761850291, -2.495065217511268],
     [107.66288796654008, -2.767442755874634]]]}},
 {'type': 'Feature',
  'properties': {'id': 'carnival-submarine-network-2-csn-2',
   'name': 'Carnival Submarine Network-2 (CSN-2)',
   'color': '#b2692e',
   'feature_id': 'carnival-submarine-network-2-csn-2-0',
   'coordinates': [-89.47684095006802, 23.929751590589426]},
  'geometry': {'type': 'MultiLineString',
   'coordinates': [[[-96.14075765764133, 19

In [ ]:
landing_points_data
# here, we try to examine the structure of the landing_points_data

[{'type': 'Feature',
  'properties': {'id': 'hulhumale-maldives',
   'name': 'Hulhumale, Maldives',
   'is_tbd': False},
  'geometry': {'type': 'Point',
   'coordinates': [73.54015885755544, 4.211942635751779]}},
 {'type': 'Feature',
  'properties': {'id': 'kulhudhufushi-maldives',
   'name': 'Kulhudhufushi, Maldives',
   'is_tbd': False},
  'geometry': {'type': 'Point',
   'coordinates': [73.07140918962199, 6.622441345756148]}},
 {'type': 'Feature',
  'properties': {'id': 'eydhafushi-maldives',
   'name': 'Eydhafushi, Maldives',
   'is_tbd': False},
  'geometry': {'type': 'Point',
   'coordinates': [73.07082325253708, 5.10323036500878]}},
 {'type': 'Feature',
  'properties': {'id': 'dhangethi-maldives',
   'name': 'Dhangethi, Maldives',
   'is_tbd': False},
  'geometry': {'type': 'Point',
   'coordinates': [72.95558895917019, 3.608093477293043]}},
 {'type': 'Feature',
  'properties': {'id': 'gan-maldives',
   'name': 'Gan, Maldives',
   'is_tbd': False},
  'geometry': {'type': 'Point'

In [ ]:
print(len(landing_points_data))
print(len(cable_data))


####################
# here, the numbers in the output of the cell represent the number of cables and the total number of landing points 

1862
667


In [ ]:
def haversine(lon1, lat1, lon2, lat2):
    R = 6371  
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    return R * c

def extract_country(name):
    if ',' in name:
        return name.split(',')[-1].strip()
    return None

def extract_city(name):
    if ',' in name:
        return name.split(',')[0].strip()
    return None

####################
# here, the haversine distance function is implemented to calculate the distance between two points on earth's surface 
# when the input is given in the form of the (lon, lat) pairs 

In [ ]:
output = []
cable_count = 0

for cable in cable_data:
    cable_id = cable["properties"].get("id")
    cable_name = cable["properties"].get("name")
    cable_color = cable["properties"].get("color")
    feature_id = cable["properties"].get("feature_id")
    cable_center_coords = cable["properties"].get("coordinates")
    cable_geometry = cable["geometry"]["coordinates"]

    cable_coords = [pt for segment in cable_geometry for pt in segment]

    seen_ids = set()
    associated_landing_points = []

    for lp in landing_points_data:
        lp_id = lp["properties"]["id"]
        lp_name = lp["properties"]["name"]
        lp_country = extract_country(lp_name)
        lp_city = extract_city(lp_name)
        lp_coords = lp["geometry"]["coordinates"]

        for coord in cable_coords:
            distance = haversine(coord[0], coord[1], lp_coords[0], lp_coords[1])
            if distance <= 5:
                if lp_id not in seen_ids:
                    associated_landing_points.append({
                        "id": lp_id,
                        "name": lp_name,
                        "city": lp_city,
                        "country": lp_country,
                        "coordinates": lp_coords
                    })
                    seen_ids.add(lp_id)
                break

    if not associated_landing_points and cable_center_coords:
        print(f" No matches from geometry for cable: {cable_name} ({cable_id}). Trying fallback using properties.coordinates...")
        for lp in landing_points_data:
            lp_id = lp["properties"]["id"]
            lp_name = lp["properties"]["name"]
            lp_country = extract_country(lp_name)
            lp_city = extract_city(lp_name)
            lp_coords = lp["geometry"]["coordinates"]

            distance = haversine(cable_center_coords[0], cable_center_coords[1], lp_coords[0], lp_coords[1])
            if distance <= 10:  
                if lp_id not in seen_ids:
                    associated_landing_points.append({
                        "id": lp_id,
                        "name": lp_name,
                        "city": lp_city,
                        "country": lp_country,
                        "coordinates": lp_coords
                    })
                    seen_ids.add(lp_id)

    output.append({
        "cable_id": cable_id,
        "cable_name": cable_name,
        "color": cable_color,
        "feature_id": feature_id,
        "center_coordinates": cable_center_coords,
        "landing_points": associated_landing_points
    })

    cable_count += 1
    print(f"Processed: {cable_name} ({cable_id}) — {len(associated_landing_points)} landing points")

with open("cable-and-landing-points.json", "w") as out_file:
    json.dump(output, out_file, indent=2)

print(f"\n Total cables processed: {cable_count}")




####################
# here, we use the cable_data and the landing_points_data to associate each cable with its landing points
# the results are saved to "cable-and-landing-points.json"

Processed: Tanjung Pandan-Sungai Kakap (tanjung-pandan-sungai-kakap) — 2 landing points
Processed: Carnival Submarine Network-2 (CSN-2) (carnival-submarine-network-2-csn-2) — 2 landing points
Processed: Groote Eylandt (groote-eylandt) — 2 landing points
Processed: Sovetskaya Gavan-Ilyinskoye (sovetskaya-gavan-ilyinskoye) — 2 landing points
Processed: Tampnet South (tampnet-south) — 2 landing points
Processed: Tampnet North (tampnet-north) — 4 landing points
Processed: Tasman Express (TX) (tasman-express-tx) — 4 landing points
Processed: Padang-Tua Pejat (padang-tua-pejat) — 2 landing points
Processed: GC-LNZ-FU Ring (gc-lnz-fu-ring) — 9 landing points
Processed: Arctic Way (arctic-way) — 3 landing points
Processed: Thailand Domestic Submarine Cable Network (TDSCN) (thailand-domestic-submarine-cable-network-tdscn) — 5 landing points
Processed: CAT Submarine Network (CSN) (cat-submarine-network-csn) — 2 landing points
Processed: ORCA (orca) — 4 landing points
Processed: ICE IV (ice-iv) —

In [ ]:
with open("cable-and-landing-points.json", "r") as cable_lp_file:
    cable_lp_data = json.load(cable_lp_file)
    
# here, we are laoding the newly created file which contains the cables and associated landing points

In [10]:
known_places = set()

for cable in cable_lp_data:
    for landing_point in cable.get("landing_points", []):
        full_name = landing_point.get("name", "")              # e.g., "Budai, Taiwan"
        country = landing_point.get("country", "").strip()     # e.g., "Taiwan"

        # Always add full name
        if full_name:
            known_places.add(full_name.strip())

        # If formatted as "City, Country", split and add city
        if "," in full_name:
            city = full_name.split(",")[0].strip()
            known_places.add(city)
        else:
            # Sometimes it's just city already, or a compound name — add directly
            known_places.add(full_name.strip())

        # Add country separately
        if country:
            known_places.add(country)


In [ ]:
print(sorted(known_places))
known_places_list = list(known_places)


#####################
# here, we try to build a list of known places using the landing_points_data to extract already known cities, countries, regions etc
# this will be useful to do some kind of fuzzy matching in case the fault data contains misspellings

['Aasiaat', 'Aasiaat, Greenland', 'Aberdeen', 'Aberdeen, United Kingdom', 'Abidjan', "Abidjan, Côte d'Ivoire", 'Abu Dhabi', 'Abu Dhabi, United Arab Emirates', 'Abu Talat', 'Abu Talat, Egypt', 'Accra', 'Accra, Ghana', 'Achnaba', 'Achnaba, United Kingdom', 'Addenbroke Island', 'Addenbroke Island, BC, Canada', 'Adelaide', 'Adelaide, SA, Australia', 'Aden', 'Aden, Yemen', 'Aeng Batu Batu', 'Aeng Batu Batu, Indonesia', 'Aethos', 'Aethos, Greece', 'Afuá', 'Afuá, Brazil', 'Agat', 'Agat, Guam', 'Agats', 'Agats, Indonesia', 'Agatti', 'Agatti, India', 'Agios Sostis', 'Agios Sostis, Greece', 'Aguimes', 'Aguimes, Canary Islands, Spain', 'Aguni', 'Aguni, Japan', 'Ahousat', 'Ahousat, BC, Canada', 'Aikerness Bay', 'Aikerness Bay, United Kingdom', 'Aitutaki', 'Aitutaki, Cook Islands', 'Ajaccio', 'Ajaccio, France', 'Ajigaura', 'Ajigaura, Japan', 'Akita', 'Akita, Japan', 'Aktau', 'Aktau, Kazakhstan', 'Akulivik', 'Akulivik, QC, Canada', 'Akutan', 'Akutan, AK, United States', 'Al Bayda', 'Al Bayda, Libya'

In [ ]:
# Load spaCy English model
nlp = spacy.load("en_core_web_sm")

# Initialize geolocator
geolocator = Nominatim(user_agent="subsea-cable-finder")

# ---- Step 1: Utilities for spelling correction ----
def correct_misspellings(candidate):
    correction = get_close_matches(candidate, known_places_list, n=1, cutoff=0.8)
    return correction[0] if correction else candidate

def clean_fault_description(text):
    words = text.split()
    corrected = [correct_misspellings(word) for word in words]
    return " ".join(corrected)

# ---- Step 2: Extraction ----
def extract_ner_locations(text):
    doc = nlp(text)
    return [ent.text.strip() for ent in doc.ents if ent.label_ in ("GPE", "LOC", "FAC")]

def extract_with_regex(text):
    patterns = [
        r"\bbetween ([A-Z][a-zA-Z\s\-]+) and ([A-Z][a-zA-Z\s\-]+)",
        r"\bfrom ([A-Z][a-zA-Z\s\-]+)",
        r"\boff the coast of ([A-Z][a-zA-Z\s\-]+)",
        r"\bnear ([A-Z][a-zA-Z\s\-]+)",
        r"\bin the vicinity of ([A-Z][a-zA-Z\s\-]+)",
        r"\bin ([A-Z][a-zA-Z\s\-]+)",
        r"\bat ([A-Z][a-zA-Z\s\-]+)",
        r"\bto ([A-Z][a-zA-Z\s\-]+)"
    ]
    matches = []
    for pattern in patterns:
        found = re.findall(pattern, text)
        if found:
            if isinstance(found[0], tuple):
                for group in found:
                    matches.extend(group)
            else:
                matches.extend(found)
    return list(set(match.strip() for match in matches))

def extract_location_candidates(description):
    cleaned_text = clean_fault_description(description)
    ner_candidates = extract_ner_locations(cleaned_text)
    regex_candidates = extract_with_regex(cleaned_text)
    candidates = list(set(ner_candidates + regex_candidates))

    # Fallback if still empty
    if not candidates:
        words = cleaned_text.split()
        fallback_candidates = [w for w in words if w in known_places]
        candidates = fallback_candidates

    return candidates

# ---- Step 3: Fallback geocoder ----
def geocode_with_retry(location, retries=3, delay=1.0):
    for attempt in range(retries):
        try:
            return geolocator.geocode(location)
        except GeocoderTimedOut:
            time.sleep(delay)
    return None

# ---- Step 4: Matcher ----
def match_candidates_to_landing_points(candidates, landing_points):
    landing_names = [lp["name"] for lp in landing_points]
    matches = []

    for candidate in candidates:
        # Step 1: Fuzzy match
        close = get_close_matches(candidate, landing_names, n=1, cutoff=0.7)
        if close:
            matched_name = close[0]
            matched_lp = next(lp for lp in landing_points if lp["name"] == matched_name)
            matches.append({
                "candidate": candidate,
                "matched_name": matched_name,
                "coordinates": matched_lp["coordinates"]
            })
            continue

        # Step 2: Substring match
        substring_match = next((lp for lp in landing_points if candidate.lower() in lp["name"].lower()), None)
        if substring_match:
            matches.append({
                "candidate": candidate,
                "matched_name": substring_match["name"],
                "coordinates": substring_match["coordinates"]
            })
            continue

        # Step 3: City match
        city_match = next((lp for lp in landing_points if candidate.lower() == lp.get("city", "").lower()), None)
        if city_match:
            matches.append({
                "candidate": candidate,
                "matched_name": city_match["name"],
                "coordinates": city_match["coordinates"]
            })
            continue

        # Step 4: Country match
        country_match = next((lp for lp in landing_points if candidate.lower() == lp["country"].lower()), None)
        if country_match:
            matches.append({
                "candidate": candidate,
                "matched_name": country_match["name"],
                "coordinates": country_match["coordinates"]
            })
            continue

        # Step 5: Fallback geocoding
        geo_result = geocode_with_retry(candidate)
        if geo_result:
            matches.append({
                "candidate": candidate,
                "matched_name": f"Geocoded: {candidate}",
                "coordinates": [geo_result.longitude, geo_result.latitude]
            })
        else:
            matches.append({
                "candidate": candidate,
                "matched_name": None,
                "coordinates": None
            })

    return matches

# ---- Step 5: Full flow ----
cable_name = "C-Lion1"
fault_description = "Gulf of Finland, southeast of Porkkalanniemi"

# Find landing points for the cable
landing_points = []
for cable in cable_lp_data:
    if cable.get("cable_name") == cable_name:
        landing_points = cable.get("landing_points", [])
        break

# Run extraction + matching
candidates = extract_location_candidates(fault_description)
matched = match_candidates_to_landing_points(candidates, landing_points)

# Final result
print(json.dumps({
    "fault_description": fault_description,
    "cable_name": cable_name,
    "extracted_candidates": candidates,
    "matched_landing_points": matched
}, indent=2))


 

#   "fault_description": "Kaliko",
#   "cable_name": "Fibralink",

#   "fault_description": "Gulf of Finland, southeast of Porkkalanniemi",
#   "cable_name": "C-Lion1"

#   "fault_description": "between Kuwait and Al-Khobar, Saudi Arabia",
#   "cable_name": "FALCON"

#   "fault_description": "near Tema",
#   "cable_name": "Africa Coast to Europe (ACE)"





##############################
# here, the region names i.e candidate names are extracted from the fault description
# then it is checked if the candidate location is a landing point of that cable too
# if yes: the landing point's coordinate are used
# if no: then, geolocator is used to fetch the coordinates for a given location

{
  "fault_description": "Gulf of Finland, southeast of Porkkalanniemi",
  "cable_name": "C-Lion1",
  "extracted_candidates": [
    "Porkkalanniemi",
    "Gulf of Finland"
  ],
  "matched_landing_points": [
    {
      "candidate": "Porkkalanniemi",
      "matched_name": "Geocoded: Porkkalanniemi",
      "coordinates": [
        24.4779271,
        60.0152418
      ]
    },
    {
      "candidate": "Gulf of Finland",
      "matched_name": "Geocoded: Gulf of Finland",
      "coordinates": [
        26.3230491,
        59.8737114
      ]
    }
  ]
}


In [ ]:
fault_records = fault_data.to_dict(orient="records")

results = []

for record in fault_records:
    cable_name = record["Cable"]
    fault_description = record["Fault Location"]

    # Find landing points for this cable
    landing_points = []
    for cable in cable_lp_data:
        if cable.get("cable_name") == cable_name:
            landing_points = cable.get("landing_points", [])
            break

    # Step 1: Extract candidates and match
    candidates = extract_location_candidates(fault_description)
    matched = match_candidates_to_landing_points(candidates, landing_points)

    # Store output
    results.append({
        "cable_name": cable_name,
        "fault_description": fault_description,
        "extracted_candidates": candidates,
        "matched_landing_points": matched
    })

# Save to JSON
with open("fault_location_candidates.json", "w") as fault_location_candidates_file:
    json.dump(results, fault_location_candidates_file, indent=2)
    
    
# here, we have done the candidate and coordinates extraction for all the given fault descriptions
# the results are saved in "fault_location_candidates.json"

In [ ]:
def detect_fault_patterns(text):
    pattern_definitions = {
        "between": r"\bbetween\s+([A-Z][a-zA-Z\s\-,()]+?)\s+(?:and|&)\s+([A-Z][a-zA-Z\s\-,()]+)",
        "near": r"\bnear\s+.+?\b",
        "close": r"\bclose\s+(to\s+)?(the\s+port\s+of\s+)?[A-Z][a-zA-Z\s\-]+",
        "off_coast": r"\b(off the coast of|offshore)\s+[\w\s'\-éèêôûîàäëïçÇÉÈÊÔÛÎÀÄËÏ]+",
        "off": r"\boff\s+(?!the coast of|shore)([A-Z][a-zA-Z\s\-]+)",
        "distance_direction": r"\b(\d{1,3}(?:,\d{3})*|\d+(?:\.\d+)?)\s*(km|kms|kilometers|miles)\s+(north|south|east|west|northeast|southeast|northwest|southwest)\s+of\s+([A-Z][a-zA-Z\s\-]+)",
        "distance": r"(\d{1,3}(?:,\d{3})*(?:\.\d+)?|\d+(?:\.\d+)?)[ ]*(km|kms|kilometers|miles)",
        "marine_zone": r"\b(?:EEZ|exclusive economic zone)\b",
        "segment": r"\bS\d+[A-Z0-9]*\.?\d*\b"
    }

    detected = []
    segments_found = []
    subpatterns = {}

    for label, pattern in pattern_definitions.items():
        matches = re.findall(pattern, text, re.IGNORECASE)

        if matches:
            detected.append(label)

            if label == "between":
                for m in matches:
                    if isinstance(m, tuple) and len(m) == 2:
                        cleaned_places = []
                        for place in m:
                            match = re.match(r"([A-Z][a-zA-Z]*(?:[-\s][A-Z][a-zA-Z]*)*)", place.strip())
                            if match:
                                cleaned_places.append(match.group(1))
                        if len(cleaned_places) == 2:
                            subpatterns["between"] = cleaned_places

            if label == "distance":
                for match in matches:
                    num = match[0]
                    unit = match[1]
                    if num and unit:
                        num = float(num.replace(',', ''))
                        if "distance" not in subpatterns:
                            subpatterns["distance"] = []
                        subpatterns["distance"].append({
                            "value": num,
                            "unit": unit.lower()
                        })

            if label == "distance_direction":
                for match in matches:
                    direction = match[2].lower()
                    if "distance_direction" not in subpatterns:
                        subpatterns["distance_direction"] = direction

            if label == "segment":
                segments_found.extend(matches)

    # Enhanced 'from' pattern
    from_match = re.search(
        r"\bfrom\s+(?:the\s+)?(?:coast|port|landing station|station|cls)?(?:\s+of)?\s*([A-Z][a-zA-Z'’\-\s]+(?:\s+[A-Z][a-zA-Z'’\-\s]+)*)",
        text, re.IGNORECASE
    )

    if from_match:
        place_raw = from_match.group(1).strip()
        place_candidates = re.findall(r"[A-Z][a-zA-Z'’\-]*(?:\s+[A-Z][a-zA-Z'’\-]*)*", place_raw)
        if place_candidates:
            subpatterns["from"] = place_candidates[-1]
            if "from" not in detected:
                detected.append("from")

    result = {
        "patterns": list(set(detected))
    }

    if segments_found:
        result["segments"] = list(set(segments_found))

    if subpatterns:
        result["subpatterns"] = subpatterns

    return result


#####################
# this cell contains a function "detect_fault_patterns" which parses the fault description and gives the key patterns and sub patterns
# for example: between, near, close, far, off coast, distance from, etc
# this will be helpful when we finally process the fault description to get the locations
# this gives a better way to approach the problem of finding coordinates for given fault descriptions

In [ ]:
for fault in fault_data['Fault Location']:
    print(fault, " ", detect_fault_patterns(fault))
    
# the output of this cell just shows how the patterns and subpatterns look like

in the Red Sea, 1,450km from Zafarana   {'patterns': ['distance', 'from'], 'subpatterns': {'distance': [{'value': 1450.0, 'unit': 'km'}], 'from': 'Zafarana'}}
Sweeden Economic Zone    {'patterns': []}
S1.9 branch   {'patterns': ['segment'], 'segments': ['S1.9']}
S9 branch, located 143km from Singapore CLS   {'patterns': ['distance', 'segment', 'from'], 'segments': ['S9'], 'subpatterns': {'distance': [{'value': 143.0, 'unit': 'km'}], 'from': 'Singapore CLS'}}
off the coast of Port Hedland   {'patterns': ['off_coast']}
between Latvian town of Ventspils to Sweden's Gotland island within Sweden's exclusive economic zone   {'patterns': ['marine_zone']}
between Taiwan and Matsu   {'patterns': ['between'], 'subpatterns': {'between': ['Taiwan', 'Matsu']}}
between Taiwan and Matsu   {'patterns': ['between'], 'subpatterns': {'between': ['Taiwan', 'Matsu']}}
close to the port of Keelung on Taiwan’s north coast   {'patterns': ['close']}
near Qatar   {'patterns': ['near']}
S1 branch between Vietnam

In [ ]:
def infer_fault_location_from_patterns(patterns, subpatterns, matched_landing_points):
    result = {}

    # Helper: get best match for a location keyword, or fallback
    def resolve_location(name=None):
        if name:
            for m in matched_landing_points:
                if name.lower() in m["candidate"].lower() and m["coordinates"]:
                    return {
                        "matched_name": m["matched_name"],
                        "coordinates": m["coordinates"]
                    }
        # Fallback to first matched landing point with coordinates
        for m in matched_landing_points:
            if m.get("coordinates"):
                return {
                    "matched_name": m["matched_name"],
                    "coordinates": m["coordinates"]
                }
        return {"matched_name": name or "Unknown", "coordinates": None}

    # 1. distance_direction has highest priority
    if "distance_direction" in patterns and "distance" in subpatterns:
        direction = subpatterns["distance_direction"]
        distances = subpatterns.get("distance", [])
        if distances:
            result["method"] = "distance_direction"
            result["direction"] = direction
            result["distance"] = distances[0]
        return result

    # 2. distance + from
    if "distance" in patterns and "from" in patterns:
        place = subpatterns.get("from")
        distances = subpatterns.get("distance", [])
        if distances:
            result["method"] = "distance_from"
            result["from"] = resolve_location(place)
            result["distance"] = distances[0]
        return result

    # 3. distance + off
    if "distance" in patterns and "off" in patterns:
        place = subpatterns.get("off")
        distances = subpatterns.get("distance", [])
        if distances:
            result["method"] = "distance_off"
            result["off"] = resolve_location(place)
            result["distance"] = distances[0]
        return result

    # 4. distance + off_coast
    if "distance" in patterns and "off_coast" in patterns:
        place = subpatterns.get("off_coast")
        distances = subpatterns.get("distance", [])
        if distances:
            result["method"] = "distance_off_coast"
            result["off_coast"] = resolve_location(place)
            result["distance"] = distances[0]
        return result

    # 5. off only
    if "off" in patterns:
        place = subpatterns.get("off")
        result["method"] = "off_only"
        result["off_only"] = resolve_location(place)
        return result

    # 6. off_coast only
    if "off_coast" in patterns:
        place = subpatterns.get("off_coast")
        result["method"] = "off_coast_only"
        result["off_coast_only"] = resolve_location(place)
        return result

    # 7. between
    if "between" in patterns:
        places = subpatterns.get("between", [])
        if places and len(places) == 2:
            result["method"] = "between"
            result["between"] = [resolve_location(places[0]), resolve_location(places[1])]
        return result

    # 8. near
    if "near" in patterns:
        place = subpatterns.get("near")
        result["method"] = "near"
        result["near"] = resolve_location(place)
        return result

    # 9. close
    if "close" in patterns:
        place = subpatterns.get("close")
        result["method"] = "close"
        result["close"] = resolve_location(place)
        return result

    # fallback
    result["method"] = "fallback"
    result["message"] = "No applicable pattern used"
    return result


#########################
# here, we have a function "infer_fault_location_from_patterns" which uses the patterns and subpatterns 
# for example, if distance is present, it also captures information such as distance in km or the place from which the distance is given

In [ ]:
with open("fault_location_candidates.json", "r") as fault_location_candidates_file:
    fault_records = json.load(fault_location_candidates_file)

# Process each record
enriched_records = []

for record in fault_records:
    fault_description = record.get("fault_description", "")
    matched_landing_points = record.get("matched_landing_points", [])
    
    # Step 1: Extract patterns and subpatterns
    pattern_result = detect_fault_patterns(fault_description)
    patterns = pattern_result.get("patterns", [])
    subpatterns = pattern_result.get("subpatterns", {})

    # Step 2: Infer fault location
    inferred_location = infer_fault_location_from_patterns(
        patterns, subpatterns, matched_landing_points
    )

    # Step 3: Add new fields to record
    record["patterns"] = patterns
    record["subpatterns"] = subpatterns
    record["inferred_location"] = inferred_location

    enriched_records.append(record)

# Save to new JSON
with open("fault_location_inference2.json", "w") as f:
    json.dump(enriched_records, f, indent=2)

print(" Enriched records saved to fault_location_inference2.json")


 Enriched records saved to faltu_fault_location_inference2.json


In [ ]:
COORD_TOLERANCE = 1e-5

def coords_match(coord1, coord2, tol=COORD_TOLERANCE):
    return abs(coord1[0] - coord2[0]) <= tol and abs(coord1[1] - coord2[1]) <= tol

def load_data():
    with open("cable-geo.json", "r") as f:
        cable_geo = json.load(f)["features"]
    with open("cable-and-landing-points.json", "r") as f:
        cable_lp = json.load(f)
    return cable_geo, cable_lp

def extract_segments(cable_feature):
    segments = []
    multiline = cable_feature["geometry"]["coordinates"]
    for i, segment_coords in enumerate(multiline):
        start = segment_coords[0]
        end = segment_coords[-1]
        segments.append({
            "segment_index": i,
            "coords": segment_coords,
            "start": start,
            "end": end
        })
    return segments

def find_segment_connections(segments):
    connections = {s["segment_index"]: set() for s in segments}
    for i, seg1 in enumerate(segments):
        for j, seg2 in enumerate(segments):
            if i == j:
                continue
            for coord in seg2["coords"]:
                if coords_match(seg1["start"], coord) or coords_match(seg1["end"], coord):
                    connections[i].add(j)
    return connections

def draw_graph(cable_name, connections, save_dir):
    G = nx.DiGraph()
    for seg, neighbors in connections.items():
        for neighbor in neighbors:
            G.add_edge(seg, neighbor)

    pos = nx.spring_layout(G, seed=42)
    plt.figure(figsize=(10, 6))
    nx.draw(G, pos, with_labels=True, node_color='lightblue', node_size=700, edge_color='gray', arrows=True)
    plt.title(f"Segment Connectivity: {cable_name}")
    
    filename = f"{save_dir}/{sanitize_filename(cable_name)}_network_graph.png"

    plt.savefig(filename)
    plt.close()

def get_segment_color(index, total):
    hue = int((index / max(1, total)) * 360)
    return f"hsl({hue}, 80%, 50%)"

def plot_segments_on_map(cable_name, segments, landing_points, save_dir):
    start_coord = segments[0]["coords"][0]
    fmap = Map(location=[start_coord[1], start_coord[0]], zoom_start=4, tiles="CartoDB positron")
    total_segments = len(segments)

    for segment in segments:
        coords = segment["coords"]
        seg_color = get_segment_color(segment["segment_index"], total_segments)

        geojson = {
            "type": "Feature",
            "geometry": {
                "type": "LineString",
                "coordinates": coords
            },
            "properties": {
                "label": f"Segment {segment['segment_index']}"
            }
        }

        GeoJson(
            geojson,
            style_function=lambda feature, color=seg_color: {
                'color': color,
                'weight': 4,
                'opacity': 0.7
            },
            tooltip=GeoJsonTooltip(
                fields=["label"],
                aliases=[""],
                style="""
                    background-color: white;
                    color: black;
                    font-family: Arial;
                    font-size: 16px;
                    padding: 5px;
                    border-radius: 4px;
                    border: 1px solid gray;
                """,
                sticky=True
            )
        ).add_to(fmap)

    if landing_points:
        for lp in landing_points:
            lat, lon = lp["coordinates"][1], lp["coordinates"][0]
            Marker(
                location=(lat, lon),
                popup=lp["name"],
                icon=folium.Icon(color="green", icon="anchor", prefix="fa")
            ).add_to(fmap)

    filename = f"{save_dir}/{sanitize_filename(cable_name)}_segments_map.html"
    fmap.save(filename)
    

def sanitize_filename(name):
    # Replace illegal characters with underscore
    return re.sub(r'[\\/*?:"<>|]', "_", name)


def main():
    cable_geo, cable_lp = load_data()
    network_data = {}

    graph_dir = "/Users/prana/Desktop/SEM6/PE/codes/faltu_network_graphs"
    map_dir = "/Users/prana/Desktop/SEM6/PE/codes/faltu_segment_maps"
    os.makedirs(graph_dir, exist_ok=True)
    os.makedirs(map_dir, exist_ok=True)

    for feature in cable_geo:
        cable_name = feature["properties"]["name"]
        print(f"Processing: {cable_name}")

        segments = extract_segments(feature)
        connections = find_segment_connections(segments)

        network_data[cable_name] = {
            "segments": segments,
            "connections": {str(k): list(v) for k, v in connections.items()}
        }

        # Find landing points
        lp_entry = next((c for c in cable_lp if c["cable_name"] == cable_name), {})
        landing_points = lp_entry.get("landing_points", [])

        draw_graph(cable_name, connections, graph_dir)
        plot_segments_on_map(cable_name, segments, landing_points, map_dir)

    with open("/Users/prana/Desktop/SEM6/PE/codes/fsegment_networks.json", "w") as f:
        json.dump(network_data, f, indent=2)

    print(" All cables processed and saved!")

if __name__ == "__main__":
    main()
    
    
#########################
# here, we check each cables' coordinate data which has been given in the form of polylines and multistrings 
# the connectivity, the segments, starting point, ending point etc are figured out
#### for this cell, we have outputs of three types:
#   1.  - we have png images for each cable in a new "network_graphs" folder
#       - each images shows the segments and the connectivity
#   2.  - we have html files for each cable in a new "segment_maps" folder
#       - it can then be used to see the cables plotted on a map with all the segments clearly visible
#   3.  - we have a json file named "segment_networks.json" that contains the  segment connectivity in terms of coordinates
#       - this will be used while trying to get the final coordinates location when we need to traverse on the cable (for notions such as x km from place Y)

FileNotFoundError: [Errno 2] No such file or directory: 'faltu-cable-and-landing-points.json'

In [ ]:
def haversine_new(lon1, lat1, lon2, lat2):
    R = 6371
    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)
    a = math.sin(delta_phi / 2) ** 2 + math.cos(phi1) * math.cos(phi2) * math.sin(delta_lambda / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

def load_segment_networks(file_path):
    with open(file_path, 'r') as f:
        return json.load(f)

def flatten_coords(segments):
    all_coords = []
    for seg in segments:
        all_coords.extend(seg.get("coords", []))
    return all_coords


def process_between_faults(fault, segment_networks):
    cable_name = fault.get("cable_name")
    cable_data = segment_networks.get(cable_name)
    inferred = fault.get("inferred_location", {})
    between_coords = inferred.get("between", [])

    if not cable_data or len(between_coords) != 2:
        return {"cable_name": cable_name, "method": "between", "final_coordinates": None, "error": "Missing cable or invalid between data"}

    lon_a, lat_a = between_coords[0]["coordinates"]
    lon_b, lat_b = between_coords[1]["coordinates"]

    all_coords = flatten_coords(cable_data["segments"])
    min_diff = float('inf')
    best_coord = None

    for lon_m, lat_m in all_coords:
        dist_am = haversine_new(lon_a, lat_a, lon_m, lat_m)
        dist_bm = haversine_new(lon_b, lat_b, lon_m, lat_m)
        diff = abs(dist_am - dist_bm)
        if diff < min_diff:
            min_diff = diff
            best_coord = [lon_m, lat_m]

    if best_coord:
        return {
            "cable_name": cable_name,
            "method": "between",
            "final_coordinates": best_coord,
            "notes": "Midpoint between two locations"
        }
    else:
        return {"cable_name": cable_name, "method": "between", "final_coordinates": None, "error": "No suitable midpoint found"}

def handle_off_only_types(fault, segment_networks):
    method = fault["inferred_location"]["method"]
    cable_name = fault["cable_name"]
    inferred = fault["inferred_location"].get(method, {}).get("coordinates")

    if not inferred:
        return {"cable_name": cable_name, "method": method, "final_coordinates": None, "error": f"No coordinates under '{method}'"}

    cable_data = segment_networks.get(cable_name)
    if not cable_data:
        return {"cable_name": cable_name, "method": method, "final_coordinates": None, "error": "Cable not found"}

    all_coords = flatten_coords(cable_data["segments"])
    filtered = [coord for coord in all_coords if haversine_new(coord[0], coord[1], inferred[0], inferred[1]) >= 100]

    if not filtered:
        return {
            "cable_name": cable_name,
            "method": method,
            "final_coordinates": None,
            "notes": "No point >100km from inferred location"
        }

    closest = min(filtered, key=lambda coord: haversine_new(coord[0], coord[1], inferred[0], inferred[1]))
    return {
        "cable_name": cable_name,
        "method": method,
        "final_coordinates": closest,
        "notes": f"Closest cable point >100km from location {inferred}"
    }

def process_distance_based_fault(fault_record, segment_data):
    def find_nearest_segment_and_point(cable_name, input_coord):
        cable_segments = segment_data.get(cable_name)
        if not cable_segments:
            raise ValueError("Cable not found")

        min_dist = float("inf")
        nearest_point = None
        best_segment = None

        for segment in cable_segments["segments"]:
            for pt in segment["coords"]:
                # dist = geodesic((pt[1], pt[0]), (input_coord[1], input_coord[0])).km
                # dist = geodesic(to_latlon(pt), to_latlon(input_coord)).km
                dist = geodesic(to_latlon(pt), input_coord).km


                if dist < min_dist:
                    min_dist = dist
                    nearest_point = tuple(pt)
                    best_segment = segment

        return best_segment, nearest_point

    def walk_segment_both_directions(coords, start_point, max_distance_km):
        start_idx = min(
            range(len(coords)),
            key=lambda i: geodesic(to_latlon(coords[i]), to_latlon(start_point)).km
        )   

        walked = [tuple(coords[start_idx])]
        total_walked = 0

        # Forward
        i = start_idx
        while i + 1 < len(coords):
            prev, curr = tuple(coords[i]), tuple(coords[i + 1])
            # dist = geodesic((prev[1], prev[0]), (curr[1], curr[0])).km
            dist = geodesic(to_latlon(prev), to_latlon(curr)).km

            if total_walked + dist > max_distance_km:
                break
            walked.append(curr)
            total_walked += dist
            i += 1

        # Backward
        i = start_idx
        while i - 1 >= 0:
            prev, curr = tuple(coords[i]), tuple(coords[i - 1])
            # dist = geodesic((prev[1], prev[0]), (curr[1], curr[0])).km
            dist = geodesic(to_latlon(prev), to_latlon(curr)).km

            if total_walked + dist > max_distance_km:
                break
            walked.insert(0, curr)
            total_walked += dist
            i -= 1

        return walked, total_walked

    def walk_all_paths(cable_name, start_coord, distance_km):
        cable_segments = segment_data[cable_name]
        segment_map = {seg["segment_index"]: seg for seg in cable_segments["segments"]}
        connections = cable_segments["connections"]
        start_segment, start_point = find_nearest_segment_and_point(cable_name, start_coord)

        queue = deque()
        results = []
        visited_states = set()

        queue.append((
            start_segment["segment_index"],
            start_point,
            distance_km,
            [start_point],
            [start_segment["segment_index"]]
        ))

        while queue:
            seg_idx, coord, remaining_km, path, segments = queue.popleft()
            seg = segment_map[seg_idx]
            coords = seg["coords"]
            walked, walked_km = walk_segment_both_directions(coords, coord, remaining_km)

            if walked_km == 0:
                results.append({
                    "final_coordinate": coord,
                    "segments_visited": segments,
                    "walked_path": [coord],
                    "total_distance_walked": 0
                })
                continue

            new_remaining = remaining_km - walked_km
            final_coord = walked[-1]
            full_path = path + walked[1:]

            results.append({
                "final_coordinate": final_coord,
                "segments_visited": segments,
                "walked_path": full_path,
                "total_distance_walked": round(distance_km - new_remaining, 3)
            })

            if new_remaining <= 0:
                continue

            neighbors = connections.get(str(seg_idx)) or connections.get(seg_idx)
            if not neighbors:
                continue

            for neighbor_idx in neighbors:
                state_key = (neighbor_idx, round(final_coord[0], 6), round(final_coord[1], 6))
                if state_key in visited_states:
                    continue
                visited_states.add(state_key)
                next_seg = segment_map[neighbor_idx]
                next_start = tuple(next_seg["coords"][0])
                queue.append((
                    neighbor_idx,
                    next_start,
                    new_remaining,
                    full_path,
                    segments + [neighbor_idx]
                ))

        return results

    method = fault_record.get("inferred_location", {}).get("method")
    if method not in ["distance_from", "distance_off", "distance_off_coast"]:
        return {
            "cable_name": fault_record.get("cable_name"),
            "method": method,
            "processed_location": None,
            "error": f"Unsupported method: {method}"
        }

    cable_name = fault_record.get("cable_name")
    inferred_location = fault_record.get("inferred_location", {})

    place_key = {
        "distance_from": "from",
        "distance_off": "off",
        "distance_off_coast": "off_coast"
    }[method]

    place_info = inferred_location.get(place_key)
    if not place_info or not place_info.get("coordinates"):
        return {"cable_name": cable_name, "method": method, "processed_location": None, "error": f"Coordinates missing in inferred_location[{place_key}]"}

    start_coords = place_info["coordinates"]
    distance_km = inferred_location.get("distance", {}).get("value")
    if distance_km is None:
        return {"cable_name": cable_name, "method": method, "processed_location": None, "error": "Distance value not provided"}


    if cable_name not in segment_data:
        return {
        "cable_name": cable_name,
        "method": method,
        "processed_location": None,
        "error": f"Cable '{cable_name}' not found in segment_data"
    }

    
    try:
        results = walk_all_paths(cable_name, start_coords[::-1], distance_km)
    except Exception as e:
        return {"cable_name": cable_name, "method": method, "processed_location": None, "error": f"Error while processing: {str(e)}"}

    if not results:
        return {"cable_name": cable_name, "method": method, "processed_location": None, "error": "Could not walk the cable for given distance"}

    best_result = max(results, key=lambda r: r["total_distance_walked"])
    return {
        "cable_name": cable_name,
        "method": method,
        "processed_location": best_result["final_coordinate"],
        "notes": f"Reached after walking {best_result['total_distance_walked']} km along connected cable segments."
    }

def to_latlon(coord):
    """Convert [lon, lat] to (lat, lon) for geopy."""
    return (coord[1], coord[0])

# pipeline

def process_fault_records(faults, segment_networks):
    processed = []

    for fault in faults:
        method = fault.get("inferred_location", {}).get("method", "")

        if method == "between":
            result = process_between_faults(fault, segment_networks)
        elif method in {"off_coast_only", "off_only", "near", "close"}:
            result = handle_off_only_types(fault, segment_networks)
        elif method in {"distance_from", "distance_off", "distance_off_coast"}:
            result = process_distance_based_fault(fault, segment_networks)
        else:
            result = {
                "cable_name": fault.get("cable_name"),
                "method": method,
                "final_coordinates": None,
                "error": f"Unsupported method: {method}"
            }

        fault["final_inference"] = result
        processed.append(fault)

    return processed


if __name__ == "__main__":
    with open("faltu_fault_location_inference2.json", "r") as f:
        fault_data = json.load(f)

    segment_networks = load_segment_networks("faltu_segment_networks.json")
    output = process_fault_records(fault_data, segment_networks)

    with open("faltu_faults_with_final_inference_temp.json", "w") as f:
        json.dump(output, f, indent=2)

    print("Processed fault records written to 'faults_with_final_inference_temp.json'")
    
    
#######################################################
    


✅ Processed fault records written to 'faults_with_final_inference_temp.json'


In [ ]:
with open("faltu_faults_with_final_inference_temp.json", "r") as f:
    faults = json.load(f)

error_count = sum(1 for fault in faults if "error" in fault.get("final_inference", {}))
print(f"Number of records with errors in final_inference: {error_count}")


Number of records with errors in final_inference: 58


In [ ]:
df = pd.read_csv(TELEGEOGRAPHY_FAULT_DATA_FILE_PATH)

# Drop rows with missing or empty Fault Location
df = df[~df["Fault Location"].isna() & (df["Fault Location"].str.strip() != "")]

# Replace NaNs in category
df["Final Inferred Category"] = df["Final Inferred Category"].fillna("NA")

# Normalize text columns for easier comparison
df["Fault Location"] = df["Fault Location"].astype(str).str.strip().str.lower()
df["Cable"] = df["Cable"].astype(str).str.strip().str.lower()

# Load the JSON file
with open("faltu_faults_with_final_inference_temp.json", "r") as f:
    faults = json.load(f)

# Enrich JSON records
for record in faults:
    fault_desc = record.get("fault_description", "").strip().lower()
    cable_name = record.get("cable_name", "").strip().lower()

    best_match = None

    for _, row in df.iterrows():
        csv_fault_loc = row["Fault Location"]
        csv_cable = row["Cable"]

        # Check both fault location and cable name match
        location_match = csv_fault_loc in fault_desc or get_close_matches(csv_fault_loc, [fault_desc], n=1, cutoff=0.9)
        cable_match = (csv_cable == cable_name)

        if location_match and cable_match:
            best_match = row
            break

    if best_match is not None:
        record["region"] = best_match["Region"]
        record["date_of_fault"] = best_match["Date of Fault"]
        record["final_inferred_category"] = best_match["Final Inferred Category"]

# Save enriched JSON
with open("faltu_faults_with_final_inference_enriched.json", "w") as f:
    json.dump(faults, f, indent=2)

print(" Enriched JSON saved to 'faults_with_final_inference_enriched.json'")


✅ Enriched JSON saved to 'faults_with_final_inference_enriched.json'


In [ ]:
import folium
import json

# Load the enriched JSON file
with open("faults_with_final_inference_enriched.json", "r") as f:
    fault_data = json.load(f)

# Initialize a map with a dark theme tile layer
m = folium.Map(location=[0, 100], zoom_start=2, tiles="CartoDB dark_matter")

# Category colors
category_colors = {
    "Aging": "orange",
    "Geopolitical": "red",
    "Human Activity": "blue",
    "Environmental": "green",
    "NA": "gray"
}

# Function to add fault marker
def add_marker(fault):
    coordinates = None
    if "final_inference" in fault:
        if "final_coordinates" in fault["final_inference"]:
            coordinates = fault["final_inference"]["final_coordinates"]
        elif "processed_location" in fault["final_inference"]:
            coordinates = fault["final_inference"]["processed_location"]
    
    if not coordinates:
        return

    category = fault.get("final_inferred_category", "NA")
    color = category_colors.get(category, "gray")

    tooltip = folium.Popup(f"""
        <b>Cable Name:</b> {fault.get('cable_name', 'N/A')}<br>
        <b>Fault Description:</b> {fault.get('fault_description', 'N/A')}<br>
        <b>Region:</b> {fault.get('region', 'N/A')}<br>
        <b>Date of Fault:</b> {fault.get('date_of_fault', 'N/A')}<br>
        <b>Category:</b> {category}
    """, max_width=300)

    # Add glowing circle marker
    folium.CircleMarker(
        location=[coordinates[1], coordinates[0]],  # lat, lon
        radius=7,
        color=color,
        fill=True,
        fill_opacity=0.85,
        popup=tooltip
    ).add_to(m)

# Add all markers
for fault in fault_data:
    add_marker(fault)

# Create a custom legend for the map
legend_html = '''
    <div style="position: fixed; 
                bottom: 30px; left: 30px; width: 150px; height: 150px; 
                background-color: rgba(255, 255, 255, 0.8); border: 2px solid black; 
                padding: 10px; font-size: 12px; z-index: 9999;">
        <b>Fault Categories</b><br>
        <i style="background-color: orange; width: 18px; height: 18px; display: inline-block;"></i> Aging<br>
        <i style="background-color: red; width: 18px; height: 18px; display: inline-block;"></i> Geopolitical<br>
        <i style="background-color: blue; width: 18px; height: 18px; display: inline-block;"></i> Human Activity<br>
        <i style="background-color: green; width: 18px; height: 18px; display: inline-block;"></i> Environmental<br>
        <i style="background-color: gray; width: 18px; height: 18px; display: inline-block;"></i> Unknown
    </div>
'''

# Add legend to map
m.get_root().html.add_child(folium.Element(legend_html))

# Save the map
m.save("telegeograph_fault_locations_map.html")
print("Map saved as 'telegeograph_fault_locations_map.html'")


Map saved as 'telegeograph_fault_locations_map_with_legend.html'
